## BPE

byte paire encoding 

```
aaabdaaabac
```

2 바이트씩 묶어서 압축

위에서는 aa 패턴이 제일 많이 나왔으므로

```
YabdYabac
```

그 다음 알파벳 순으로 제일 많이 나온 ab

```
YXdYXac
```

그 다음 YX

```
ZdZac
```

이렇게 데이터 사전 크기가 원하는 크기가 될 때 까지 계속 반복하며 압축

데이터 사전

```
a, b, c, d, Y, X, Z
```

ex)

| token | count |
|:-----:|:-----:|
| hug | 10 |
| pug | 5 |
| pun | 12 |
| bun | 4 |
| hugs | 5 |

pre-tokenized 결과에서 문자 로 분리

| token | count |
|:-----:|:-----:|
| h, u, g | 10 |
| p, u, g | 5 |
| p, u, n | 12 |
| b, u, n | 4 |
| h, u, g, s | 5 |

여기서 페어 별로 재정렬

| token | count | sum |
|:-----:|:-----:|:---:|
| h, u | 10 + 5 | 15 |
| u, g | 10 + 5 + 5 | 20 |
| p, u | 5 + 12 | 17 |
| ~~u, g~~ | 5 | |
| ~~p, u~~ | 12 | |
| u, n | 12 + 4 | 16 |
| b, u | 4 | 4 |
| ~~u, n~~ | 4 | |
| ~~h, u~~ | 5 | |
| ~~u, g~~ | 5 | |
| g, s | 5 | 5 |

u, g 패턴이 가장 많이 나왔으므로 어휘 사전에 추가

(b, h, u, g, s, n, p, `ug`)

위 어휘 사전을 참고하여 다시 문자별 분리

| token | count |
|:-----:|:-----:|
| h, ug | 10 |
| p, ug | 5 |
| p, u, n | 12 |
| b, u, n | 4 |
| h, ug, s | 5 |

다시 BPE 진행

| token | count | sum |
|:-----:|:-----:|:---:|
| h, ug | 10 + 5 | 15 |
| p, ug | 5 | 5 |
| p, u | 12 | 12 |
| u, n | 12 + 4 | 16 |
| b, u | 4 | 4 |
| ~~u, n~~ | 4 | |
| ~~h, ug~~ | 5 | |
| ug, s | 5 | 5 |

가장 많이 나온 un 어휘 사전에 추가

(b, h, u, g, s, n, p, ug, `un`)

위 어휘 사전을 참고하여 다시 문자별 분리

| token | count |
|:-----:|:-----:|
| h, ug | 10 |
| p, ug | 5 |
| p, un | 12 |
| b, un | 4 |
| h, ug, s | 5 |

| token | count | sum |
|:-----:|:-----:|:---:|
| h, ug | 10 + 5 | 15 | 
| p, ug | 5 | 5 |
| p, un | 12 | 12 |
| b, un | 4 | 4 |
| ~~h, ug~~ | 5 | |
| ug, s | 5 | 5 |

가장 많이 나온 hug 어휘 사전에 추가

(b, h, u, g, s, n, p, ug, un, `hug`)

위 과정을 원하는 어휘 사전 길이가 될 때 까지 반복

## BPE 토큰화

input == `pug bug mug`

공백 기준 pre-tokenize > `pug, bug, mug`

위 어휘 사전을 만들면서 나온 병합 우선순위 정보 참고하여 분리

병합 우선순위

1. u g -> ug
2. u n -> un
3. h ug -> hug

pug 는 p, u, g 로 나눌 수 있음,

병합 가능한 u, g 조합으로 인해 p, ug 로 변환, 이 후 병합 가능한 내용 없으면 종료

bug 도 같음

mug 경우엔 없는 토큰 m 이 존재 하기 때문에 `<unk>` 처리

p, ug, b, ug, `<unk>`, ug

## Wordpiece

BPE 와 유사하지만 

병합된 내용이 나올 확률 분에 각각으로 나누어져 나올 확률로 계산하여 판단

문자 단위로 진행하기 때문에 읽을 수 있는 문자 형태로 저장됨

## Tokenizer coding

네이버 영화 리뷰 데이터 활용

### GPT (BPE) Tokenizer

In [16]:
from Korpora import Korpora
import os
from tokenizers import ByteLevelBPETokenizer, BertWordPieceTokenizer
from transformers import GPT2Tokenizer, BertTokenizer

In [2]:
nsmc = Korpora.load('nsmc', force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 18.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 16.7MB/s]                            


In [3]:
nsmc.train.get_all_texts()[0], nsmc.train.get_all_labels()[0]

('아 더빙.. 진짜 짜증나네요 목소리', 0)

In [4]:
rootDir = !pwd
rootDir[0]

'/tf/notebooks/NLP'

In [5]:
def saveData(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

In [6]:
saveData(f'{rootDir[0]}/train.txt', nsmc.train.get_all_texts())
saveData(f'{rootDir[0]}/test.txt', nsmc.test.get_all_texts())

In [7]:
os.makedirs(f'{rootDir[0]}/bpe-tokenized', exist_ok=True)

In [8]:
byteBpe = ByteLevelBPETokenizer()
byteBpe.train(
    files=[f'{rootDir[0]}/train.txt', f'{rootDir[0]}/test.txt'],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
byteBpe.save_model(f'{rootDir[0]}/bpe-tokenized')

['/tf/notebooks/NLP/bpe-tokenized/vocab.json',
 '/tf/notebooks/NLP/bpe-tokenized/merges.txt']

### BERT (wordpiece) Tokenizer

In [9]:
os.makedirs(f'{rootDir[0]}/wordpiece-tokenized', exist_ok=True)

In [10]:
wordpiece = BertWordPieceTokenizer(lowercase=False)
wordpiece.train(
    files=[f'{rootDir[0]}/train.txt', f'{rootDir[0]}/test.txt'],
    vocab_size=10000
)
wordpiece.save_model(f'{rootDir[0]}/wordpiece-tokenized')

['/tf/notebooks/NLP/wordpiece-tokenized/vocab.txt']

## Test Sentences

In [13]:
sentences = [
    nsmc.train.get_all_texts()[0],
    nsmc.train.get_all_texts()[1],
    nsmc.train.get_all_texts()[2]
]
sentences

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다']

### GPT (BPE) Input

In [11]:
bpeTokenizer = GPT2Tokenizer.from_pretrained(f'{rootDir[0]}/bpe-tokenized')
bpeTokenizer.pad_token = '[PAD]'

file /tf/notebooks/NLP/bpe-tokenized/config.json not found


In [14]:
bpeTokenizedSentences = [bpeTokenizer.tokenize(sentence) for sentence in sentences]
bpeTokenizedSentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ëĦĪë¬´',
  'ìŀ¬ë',
  '°',
  'ĵ',
  'ìĹĪëĭ¤',
  'ê·¸ëŀĺìĦľ',
  'ë³´ëĬĶ',
  'ê²ĥìĿĦ',
  'ì¶Ķì²ľ',
  'íķľëĭ¤']]

In [15]:
batchInput = bpeTokenizer(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True
)
batchInput

{'input_ids': [[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0], [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292], [694, 477, 109, 242, 643, 3965, 1067, 2285, 2716, 969, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]}

input_ids : 각 분리된 토크의 인덱스 번호, 어휘 집합 vocab.json 에서 확인 가능 (인덱싱 작업)

문자 최대 길이인 12 보다 작으면 padding 값으로 0이 채워짐

크면 잘리는 옵션인 truncation 적용

attention_mask: 일반 토큰, 패딩 토큰 위치 정보

### BERT (wordpiece) Input

In [17]:
bertTokenizer = BertTokenizer.from_pretrained(
    f'{rootDir[0]}/wordpiece-tokenized',
    do_lower_case=False
)

file /tf/notebooks/NLP/wordpiece-tokenized/config.json not found


In [18]:
bertTokenizedSentences = [bertTokenizer.tokenize(sentence) for sentence in sentences]
bertTokenizedSentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['[UNK]']]

In [19]:
batchInput = bertTokenizer(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True
)
batchInput

{'input_ids': [[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0], [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1025, 16, 3], [2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}